In [ ]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas.io.sql as psql
import psycopg2 as pg

from keras.models import Sequential
from keras.layers import Dense
from keras.engine.saving import model_from_json
from keras.layers import SimpleRNN, Activation

from sklearn import model_selection

from imblearn.over_sampling import SMOTE

/Users/ca/PycharmProjects/JupyterNoteBookExample/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Using TensorFlow backend.


In [3]:
with pg.connect(database='chi-navi',
                host='localhost',
                user='postgres',
                port=5432) as conn:
    all_season_sql = \
        "with spot_bus_stop as( " \
        "select " \
        "p.bus_stop_id,s.spot_id " \
        "from " \
        "(select bus_stop_id,latlon::point as latlon from bus_stop) as p, " \
        "spot s " \
        "where " \
        "ST_Covers( " \
        "ST_Buffer( " \
        "ST_POINT(s.area[0], s.area[1])::geography,400), " \
        "ST_POINT(p.latlon[1], p.latlon[0]) " \
        ") " \
        "), " \
        "delay_info as( " \
        "with route_bus_stop_list as( " \
        "select " \
        "t.task_id,r.route_id,b.bus_stop_id,rbs.ordinal " \
        "from " \
        "bus_stop b,route r,route_bus_stop rbs,task t " \
        "where " \
        "b.bus_stop_id = rbs.bus_stop_id " \
        "and r.route_id = rbs.route_id " \
        "and t.route_id = r.route_id " \
        "order by r.route_id,rbs.ordinal " \
        "), " \
        "delay_list as( " \
        "with stddev as( " \
        "select " \
        "stddev_samp(extract(epoch from (collide_at - departure_time))) as stddev " \
        "from " \
        "task_delay " \
        "where " \
        "(collide_at - departure_time) > '00:00:00' " \
        "), " \
        "delay_avg as( " \
        "select " \
        "avg((extract(epoch from (collide_at - departure_time)))) as delay_avg " \
        "from " \
        "task_delay " \
        "where " \
        "(collide_at - departure_time) > '00:00:00' " \
        ") " \
        "select " \
        "td.task_id,td.departure_time,td.bus_stop_id,td.submit_in,td.collide_at " \
        ",extract(epoch from (td.collide_at - td2.departure_time)) as delay " \
        "from " \
        "task_delay td,delay_avg da,stddev std,task_detail td2 " \
        "where " \
        "(extract(epoch from (td.collide_at - td2.departure_time)) between " \
        "da.delay_avg - std.stddev * 3 and da.delay_avg + std.stddev * 3) " \
        "and extract(epoch from (td.collide_at - td2.departure_time)) > 0 " \
        "and td.task_id = td2.task_id " \
        "and td.bus_stop_id = td2.bus_stop_id " \
        "and td.departure_time = td2.departure_time " \
        ") " \
        "select " \
        "distinct r.route_id,r.bus_stop_id,r.ordinal,d.collide_at, " \
        "d.task_id,d.delay,d.submit_in,d.departure_time,date_part('HOUR' , d.departure_time) as time " \
        "from " \
        "delay_list d, route_bus_stop_list r " \
        "where " \
        "d.bus_stop_id = r.bus_stop_id " \
        "and r.task_id = d.task_id " \
        "), " \
        "access_info as( " \
        "select " \
        "count(distinct session_id) as access_num,browsing_at::date as date, " \
        "date_part('HOUR' , browsing_at) as time, " \
        "case when " \
        "extract(dow from browsing_at::date) = 0 or extract(dow from browsing_at::date) = 6 then 0 " \
        "else 1 " \
        "end as youbi " \
        "from " \
        "user_browsing_log ubl " \
        "group by date,time " \
        ") " \
        "select " \
        "case " \
        "when delay between 0 and 60 then 0 " \
        "when delay between 61 and 120 then 1 " \
        "when delay between 121 and 180 then 2 " \
        "when delay between 181 and 240 then 3 " \
        "when delay between 241 and 300 then 4 " \
        "when delay between 301 and 360 then 5 " \
        "when delay between 361 and 420 then 6 " \
        "when delay between 421 and 480 then 7 " \
        "when delay between 481 and 540 then 8 " \
        "when delay > 540 then 9 " \
        "end as delay, " \
        "ai.time, " \
        "ai.access_num,ai.youbi, " \
        "case " \
        "when submit_in = '2017-07-23' then 1 /* 航空祭 */  " \
        "when (submit_in between '2018-01-26' and '2018-02-18') or " \
        "(submit_in between '2017-01-27' and '2017-02-19') " \
        "then 2/* ひょうとう祭り */ " \
        "when submit_in between '2017-07-06' and '2017-07-09' then 3/* セガサミーカップ */ " \
        "else 0 " \
        "end as event, " \
        "case " \
        "when di.route_id in (13,14,23,24,55,56,58,59,60,61,62,63,64,67,68,69,70,71,74 " \
        ",78,79,80,81,82,91,92,93,94,95,96,97,98,99,106,107,108,109,11) " \
        "then 1 " \
        "else 0 " \
        "end as destination, " \
        "di.bus_stop_id, " \
        "s.spot_id " \
        "from " \
        "spot_bus_stop s, " \
        "delay_info di, " \
        "access_info ai " \
        "where " \
        "di.time = ai.time " \
        "and di.submit_in = ai.date " \
        "and s.bus_stop_id = di.bus_stop_id "


In [4]:
sql_result = psql.read_sql(all_season_sql, conn)

In [16]:
maxLen = 100
length_of_sequence = 2 * len(sql_result)
X = np.zeros((len(sql_result), maxLen, 1), dtype=float)
Y = np.zeros((len(sql_result), 1), dtype=float)



3030004


In [ ]:

N_train = int(len(sql_result) * 0.9)
N_validation = len(sql_result) - N_train

X_train, X_validation, Y_train, Y_validation = \
    model_selection.train_test_split(X, Y, test_size=N_validation)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

model = Sequential()

model.add(SimpleRNN(units=4, kernel_initializer='he_normal', input_shape=(maxLen, 5)))
model.add(Dense(4, kernel_initializer='he_normal'))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error', 
              optimizer=optimizer)

In [ ]:
epochs = 500
batch_size = 10

model.fit(X_train, Y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(X_validation, Y_validation),
          callbacks=[EarlyStopping(monitor='loss', patience=10, verbose=1)])
